In [1]:
import numpy as np
import pandas as pd
from math import isnan
from datetime import datetime,date
# Saved Tiingo API Token in 'token' variable in apiToken.py file
from apiToken import token

In [2]:
# Getting prices from Tiingo
def getPrice(token,ticker,startDate,endDate):
    return pd.read_csv('https://api.tiingo.com/tiingo/daily/'+ticker+'/prices?startDate='+startDate+'&endDate='+endDate+'&token='+token+'&format=csv')
today = str(date.today())
bndPrice = getPrice(token,'bnd','2019-09-30',today)
bndxPrice = getPrice(token,'bndx','2019-09-30',today)
vtiPrice = getPrice(token,'vti','2019-09-30',today)
vxusPrice = getPrice(token,'vxus','2019-09-30',today)
vtiPrice['returns'] = (vtiPrice.adjClose)/vtiPrice.adjClose.shift(1) - 1
vxusPrice['returns'] = (vxusPrice.adjClose)/vxusPrice.adjClose.shift(1) - 1
bndPrice['returns'] = (bndPrice.adjClose)/bndPrice.adjClose.shift(1) - 1
bndxPrice['returns'] = (bndxPrice.adjClose)/bndxPrice.adjClose.shift(1) - 1
bndPrice = bndPrice.loc[:,['date','returns','adjClose']]
bndxPrice = bndxPrice.loc[:,['date','returns','adjClose']]
vtiPrice = vtiPrice.loc[:,['date','returns','adjClose']]
vxusPrice = vxusPrice.loc[:,['date','returns','adjClose']]

In [3]:
# Reading saved data (Market cap is in billions)
vti = pd.read_csv('Data/vti_MC_Data.csv')
vxus = pd.read_csv('Data/vxus_MC_Data.csv')
bnd = pd.read_csv('Data/bnd_MC_Data.csv')
bndx = pd.read_csv('Data/bndx_MC_Data.csv')

In [4]:
df = vtiPrice.rename(columns={"returns":"VTI_Return","adjClose":"VTI_Price"}).merge(vxusPrice.rename(columns={"returns":"VXUS_Return","adjClose":"VXUS_Price"}),on='date',how='outer').merge(bndPrice.rename(columns={"returns":"BND_Return","adjClose":"BND_Price"}),on='date',how='outer').merge(bndxPrice.rename(columns={"returns":"BNDX_Return","adjClose":"BNDX_Price"}),on='date',how='outer').merge(vti.rename(columns={"Market Cap":"VTI_MarketCap"}),on='date',how='left').merge(vxus.rename(columns={"Market Cap":"VXUS_MarketCap"}),on='date',how='left').merge(bnd.rename(columns={"Market Cap":"BND_MarketCap"}),on='date',how='left').merge(bndx.rename(columns={"Market Cap":"BNDX_MarketCap"}),on='date',how='left').sort_values(by='date').reset_index(drop=True)

In [5]:
etfs = ['VTI','VXUS','BND','BNDX']
for etf in etfs:
    list = []
    i = 0
    value = 999
    for mc,ret in zip(df[etf+'_MarketCap'],df[etf+'_Return']):
        if i>0:
            value = (ret+1) * list[i-1]
        list.append(mc) if not isnan(mc) else list.append(value)
        i+=1
    df[etf+'_MarketCap'] = list

In [6]:
df['VTI_weight'] = df['VTI_MarketCap']/(df.VTI_MarketCap+df.VXUS_MarketCap+df.BND_MarketCap+df.BNDX_MarketCap)
df['VXUS_weight'] = df['VXUS_MarketCap']/(df.VTI_MarketCap+df.VXUS_MarketCap+df.BND_MarketCap+df.BNDX_MarketCap)
df['BND_weight'] = df['BND_MarketCap']/(df.VTI_MarketCap+df.VXUS_MarketCap+df.BND_MarketCap+df.BNDX_MarketCap)
df['BNDX_weight'] = df['BNDX_MarketCap']/(df.VTI_MarketCap+df.VXUS_MarketCap+df.BND_MarketCap+df.BNDX_MarketCap)

#Enter starting amount
startingAmount = 10000

portfolioVal = [0 for i in range(len(df))]
portfolioVal[0] = startingAmount
df['Portfolio_Value'] = portfolioVal

In [7]:
i=0
list2 = []
portfolioRet = 1
for vtiR,vxusR,bndR,bndxR,vtiW,vxusW,bndW,bndxW,portfolioValue in zip(df.VTI_Return,df.VXUS_Return,df.BND_Return,df.BNDX_Return,df.VTI_weight.shift(1),df.VXUS_weight.shift(1),df.BND_weight.shift(1),df.BNDX_weight.shift(1),df.Portfolio_Value):
    if i>0:
        portfolioRet = (vtiR*vtiW+vxusR*vxusW+bndR*bndW+bndxR*bndxW)+1
    list2.append(portfolioValue) if portfolioValue != 0 else list2.append(list2[i-1]*portfolioRet)
    i+=1
df['Portfolio_Value'] = list2

In [8]:
df

,date,VTI_Return,VTI_Price,VXUS_Return,VXUS_Price,BND_Return,BND_Price,BNDX_Return,BNDX_Price,VTI_MarketCap,VXUS_MarketCap,BND_MarketCap,BNDX_MarketCap,VTI_weight,VXUS_weight,BND_weight,BNDX_weight,Portfolio_Value
0,2019-09-30,NaN,148.817031,NaN,50.677485,NaN,82.399983,NaN,56.935017,29401.430000,23547.228000,22512.530000,19421.010000,0.309873,0.248173,0.237268,0.204685,10000.000000
1,2019-10-01,-0.013046,146.875511,-0.009098,50.216424,0.000967,82.479676,-0.001668,56.840077,29017.848430,23332.996541,22534.302867,19388.625144,0.307804,0.247503,0.239030,0.205663,9935.875746
2,2019-10-02,-0.016238,144.490496,-0.015628,49.431639,0.002253,82.665507,0.000000,56.840077,28546.646704,22968.347250,22585.073856,19388.625144,0.305349,0.245680,0.241581,0.207390,9853.133140
3,2019-10-03,0.008117,145.663292,0.007740,49.814222,0.003313,82.939363,0.003749,57.053179,28778.353338,23146.113779,22659.894262,19461.315968,0.306004,0.246116,0.240946,0.206935,9911.835869
4,2019-10-04,0.013058,147.565391,0.006696,50.147755,0.001651,83.076291,0.000170,57.062866,29154.146450,23301.089728,22697.304465,19464.620096,0.308127,0.246267,0.239886,0.205720,9972.066703
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,2020-09-03,-0.034761,174.940000,-0.021751,53.070000,0.000113,88.750000,-0.000345,58.000000,34052.494017,24814.188928,23706.294015,20991.580699,0.328805,0.239601,0.228904,0.202691,10881.309962
236,2020-09-04,-0.008517,173.450000,0.002073,53.180000,-0.005859,88.230000,-0.001724,57.900000,33762.461914,24865.622144,23567.395166,20955.388318,0.327311,0.241061,0.228475,0.203153,10837.844412
237,2020-09-08,-0.028077,168.580000,-0.014291,52.420000,0.001473,88.360000,0.001382,57.980000,32814.504638,24510.265378,23602.119878,20984.342223,0.321991,0.240506,0.231595,0.205908,10707.598529
238,2020-09-09,0.019575,171.880000,0.016978,53.310000,-0.000679,88.300000,-0.000345,57.960000,33456.857617,24926.406854,23586.093088,20977.103747,0.324993,0.242130,0.229110,0.203767,10816.367880
